# Projekt zaliczeniowy - Opis obrazu

### Skład zespołu:

- Julia May 150653
- Patryk Jedlikowski 136723
- Mikołaj Sienkiewicz 136309

# Importy

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import os
import sys
import glob
import random
from pprint import pprint
from ipywidgets import Video
from PIL import Image
from PIL.ExifTags import TAGS

# Zbiór danych

Przygotowany zbiór danych prezentujących fragmenty rozgrywek bilardowych znajduje się w repozytorium GitHub: https://github.com/jedlin21/WK/tree/master/billard/data

# Przydatne funkcje

In [18]:
def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(PIL.Image.fromarray(a))

def create_tracker(tracker_type):
  tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

  if tracker_type == 'BOOSTING':
      return cv2.TrackerBoosting_create()
  if tracker_type == 'MIL':
      return cv2.TrackerMIL_create()
  if tracker_type == 'KCF':
      return cv2.TrackerKCF_create()
  if tracker_type == 'TLD':
      return cv2.TrackerTLD_create()
  if tracker_type == 'MEDIANFLOW':
      return cv2.TrackerMedianFlow_create()
  if tracker_type == 'GOTURN':
      return cv2.TrackerGOTURN_create()
  if tracker_type == 'MOSSE':
      return cv2.TrackerMOSSE_create()
  if tracker_type == "CSRT":
      return cv2.TrackerCSRT_create()

def draw_bbox(frame, bbox, color=(255, 255, 255)):
  p1 = (int(bbox[0]), int(bbox[1]))
  p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
  cv2.rectangle(frame, p1, p2, color, 2, 1)

# Przykładowe filmy

In [19]:
video_path_11 = './data/11.mp4'
video_path_22 = './data/22.mp4'

In [20]:
def display_example_clip(video_path):
    free_kick = cv2.VideoCapture(video_path)
    if free_kick.isOpened():
        print('Film wczytany!')

    free_kick_width = int(free_kick.get(3))
    free_kick_height = int(free_kick.get(4))

    print(free_kick_height, free_kick_width)

    free_kick_fps = free_kick.get(cv2.CAP_PROP_FPS)
    print(free_kick_fps)

In [21]:
display_example_clip(video_path_11)
Video.from_file(video_path_11)

Film wczytany!
1080 1920
30.0


Video(value=b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00mp41isom\x00\x00\x00(uuid\\\xa7\x08\xfb2\x8eB\x05\xa8ae\…

In [22]:
display_example_clip(video_path_22)
Video.from_file(video_path_22)

Film wczytany!
1080 1920
30.0


Video(value=b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00mp41isom\x00\x00\x00(uuid\\\xa7\x08\xfb2\x8eB\x05\xa8ae\…

# Wykorzystane metody

W celu śledzenia ruchu bil na filmie wykorzystaliśmy dwa różne podejścia, jedno oparte na trackerze CSRT, a drugie na Optical Flow. W pierwszym z nich zaimplementowaliśmy również wykrywanie wpadnięcia bili do łuzy.

Podejście oparte na trackerze CSRT:

1. W celu wykrycia początkowego ułożenia bil na stole najpierw dokonujemy transformacji do przestrzeni barw Grayscale, a następnie stosujemy trasformatę Hougha w celu wykrycia okręgów. Nakładamy również ograniczenia na współrzędne X oraz Y, filtrując uzyskane wyniki, aby mieć pewność, że wykryte okręgi znajdują się w obrębie stołu bilardowego.
2. Tworzymy osobny tracker CSRT dla każdej wykrytej bili.
3. Definiujemy parametry określające szerkość i wysokość łuz oraz współrzędne lewych górnych narożników dla wszystkich łuz.
4. Wczytując kolejne klatki filmu iterujemy po wszystkich trackerach i uaktualniamy je a następnie sprawdzamy dla każdego trackera i każdej bili oraz każdego trackera i każdej łuzy czy bila nie znalazła się od nich w odległości mniejszej niż hiperparamter diameter, co oznacza zderzenie bil lub wpadnięcie bili do łuzy, odpowiednio. Jeśli zdarzenie takie nastąpiło zapisujemy na filmie odpowiedni bounding box (w przypadku zderzenia bil biały kwadrat w lewym górnym rogu ekranu zmienia kolor na czarny w momencie zderzenia, a w przypadku wpadnięcia bili do łuzy w miejscu danej łuzy pojawia się czarny kwadrat zaraz po wystąpieniu zdarzenia).

# Śledzenie ruchu bil i wpadnięć do łuz z wykorzystaniem trackera CSRT

Przykładowe wyniki przetwarzania z wykorzystaniem tej metody znajdują się w repozytorium GitHub: https://github.com/jedlin21/WK/tree/master/billard/processed (filmy movie11 i movie22)

In [8]:
clips_to_process = [11, 22]

In [9]:
for s in clips_to_process:
    
    video_path = f'./data/{s}.mp4' 
    write_path = f'./processed/movie{s}.avi'

    free_kick = cv2.VideoCapture(video_path)
    if free_kick.isOpened():
        print('Film wczytany!')

    free_kick_width = int(free_kick.get(3))
    free_kick_height = int(free_kick.get(4))

    print(free_kick_height, free_kick_width)

    free_kick_fps = free_kick.get(cv2.CAP_PROP_FPS)
    print(free_kick_fps)

    free_kick.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, frame = free_kick.read()

    # bounding boxes dla początkowych pozycji kul bilardowych
    old_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    circles = cv2.HoughCircles(old_gray,cv2.HOUGH_GRADIENT,1,18,param1=80,param2=10,minRadius=10,maxRadius=20)
    circles = np.uint16(np.around(circles))
    x = circles[circles[...,0]>170]
    x = x[x[...,0]<1760]
    x = x[x[...,1]>135]
    x = x[x[...,1]<920]
    circles = x.reshape(1, x.shape[0], 3)

    balls = []
    for circle in circles[0, :]:
        balls.append((circle[0]-circle[2]-10, circle[1]-circle[2]-10, 2*circle[2]+20, 2*circle[2]+20))

    balls_tracker = []
    for ball_box in balls:
        balls_tracker.append(create_tracker('CSRT'))
        balls_tracker[-1].init(frame, ball_box)

    free_kick_track = cv2.VideoWriter(write_path, cv2.VideoWriter_fourcc(*'DIVX'), free_kick_fps, (free_kick_width, free_kick_height))

    free_kick.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    while free_kick.isOpened():
        
        ret, frame = free_kick.read()

        if ret:
            
            frame[...,0] = cv2.GaussianBlur(frame[...,0],(3,3),0)
            frame[...,1] = cv2.GaussianBlur(frame[...,1],(3,3),0)
            frame[...,2] = cv2.GaussianBlur(frame[...,2],(3,3),0)

            frame[0:100,0:100,:]=255
            
            # hiperparametr definiujący jak daleko od siebie muszą znajdować się składowe X i Y lewych górnych 
            # narożników bil, które są sprawdzane pod kątem kolizji
            diameter=25
            
            # szerkość i wysokość łuz
            pocket_size=50
            
            # współrzędne lewych górnych narożników łuz
            pockets=[(90,140),(90,950),(90,1750),(900,140),(900,950),(900,1750)]

            for x,tracker in enumerate(balls_tracker):
                
                ok, bbox = tracker.update(frame)
                MovingX=bbox[0]
                MovingY=bbox[1]
                
                # sprawdzenie czy nastąpiła kolizja
                for i,second in enumerate(balls):
                    
                    StoppedX=second[0]
                    StoppedY=second[1]
                    
                    if i!=x:
                        
                        if (MovingX-diameter<StoppedX and MovingY-diameter<StoppedY and MovingX>StoppedX and MovingY>StoppedY) \
                        or (MovingX-diameter<StoppedX and MovingY+diameter>StoppedY and MovingX>StoppedX and MovingY<StoppedY) \
                        or (MovingX+diameter>StoppedX and MovingY+diameter>StoppedY and MovingX<StoppedX and MovingY<StoppedY) \
                        or (MovingX+diameter>StoppedX and MovingY-diameter<StoppedY and MovingX<StoppedX and MovingY>StoppedY):
                            frame[0:100,0:100,:]=0
                            
                # sprawdzenie czy bila wpadła do łuzy
                for i,second in enumerate(pockets):
                    
                    StoppedX=second[1]
                    StoppedY=second[0]
                    
                    if i!=x:
                        
                        if (MovingX-diameter<StoppedX and MovingY-diameter<StoppedY and MovingX>StoppedX and MovingY>StoppedY) \
                        or (MovingX-diameter<StoppedX and MovingY+diameter>StoppedY and MovingX>StoppedX and MovingY<StoppedY) \
                        or (MovingX+diameter>StoppedX and MovingY+diameter>StoppedY and MovingX<StoppedX and MovingY<StoppedY) \
                        or (MovingX+diameter>StoppedX and MovingY-diameter<StoppedY and MovingX<StoppedX and MovingY>StoppedY):
                            frame[second[0]:second[0]+pocket_size,second[1]:second[1]+pocket_size,:]=0

                if ok: draw_bbox(frame, bbox, (0, 255, 0))

            free_kick_track.write(frame)
            
        else:
            break

    free_kick_track.release()

Film wczytany!
1080 1920
30.0
Film wczytany!
1080 1920
30.0


In [24]:
# konwersja przetworzonych klipów do formatu .mp4
for i in clips_to_process:
    !ffmpeg -hide_banner -loglevel error -i "./processed/movie{i}.avi" -y "./processed/movie{i}.mp4"

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


In [28]:
Video.from_file(f'./processed/movie11.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\r\x1dmoov\x00\x00\x00lmvhd\x00\x00…

In [29]:
Video.from_file(f'./processed/movie22.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\rUmoov\x00\x00\x00lmvhd\x00\x00\x0…

# Śledzenie ruchu bil z wykorzystaniem Optical Flow

Podejście oparte na Optical Flow:

1. W celu wykrycia początkowego ułożenia bil na stole wykonujemy podobne kroki jak w podejściu wykorzystującym tracker CSRT, korzystając z transformaty Hougha, służącej do wykrywania okręgów.
2. Wczytując kolejne klatki filmu wywołujemy dla każdej z nich funkcję z OpenCV obliczającą Optical Flow pomiędzy poprzednią i aktualną klatką filmu, która zwraca nowe pozycje śledzonych bil. 
3. Oznaczamy na filmie aktualne pozycje bil oraz trajektorie, które przebyły.

Przykładowy wynik przetwarzania z wykorzystaniem tej metody znajduje się w repozytorium GitHub: https://github.com/jedlin21/WK/tree/master/billard/processed (film movie_optical)

In [24]:
video_path = './data/11.mp4' 
write_path = './processed/movie_optical.avi'

In [25]:
traffic = cv2.VideoCapture(video_path)
if traffic.isOpened():
    print('Film wczytany!')

traffic_width = int(traffic.get(3))
traffic_height = int(traffic.get(4))

print(traffic_height, traffic_width)

traffic_fps = traffic.get(cv2.CAP_PROP_FPS)
print(traffic_fps)

Film wczytany!
1080 1920
30.0


In [26]:
Video.from_file(video_path)

Video(value=b'\x00\x00\x00\x18ftypmp42\x00\x00\x00\x00mp41isom\x00\x00\x00(uuid\\\xa7\x08\xfb2\x8eB\x05\xa8ae\…

In [13]:
feature_params = dict(
                      maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7
                     )

lk_params = dict(
                 winSize=(15,15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
                )

color = np.random.randint(0,255,(100,3))

In [14]:
# startowe pozycje bil na filmie
traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)
ret, old_frame = traffic.read()

old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
circles = cv2.HoughCircles(old_gray,cv2.HOUGH_GRADIENT,1,18,param1=80,param2=10,minRadius=10,maxRadius=20)
circles = np.uint16(np.around(circles))
x = circles[circles[...,0]>170]
x = x[x[...,0]<1760]
x = x[x[...,1]>135]
x = x[x[...,1]<920]
circles = x.reshape(1, x.shape[0], 3)
p0 = circles[...,:2].reshape(circles.shape[1],1,2).astype('float')
p0 = p0.astype('float32')
mask = np.zeros_like(old_frame)

In [15]:
# śledzenie ruchu bil
traffic_optical_flow = cv2.VideoWriter(write_path, cv2.VideoWriter_fourcc(*'DIVX'), traffic_fps, (traffic_width, traffic_height))
traffic.set(cv2.CAP_PROP_POS_FRAMES, 0)

while traffic.isOpened():
    
    ret, frame = traffic.read()
    
    if ret:
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.GaussianBlur(frame_gray,(3,3),0)
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        
        if p1 is not None:
            good_new = p1[st==1]
            good_old = p0[st==1]

        for i,(new,old) in enumerate(zip(good_new, good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (int(a),int(b)),(int(c),int(d)), color[i].tolist(), 2)
            frame = cv2.circle(frame,(int(a),int(b)),5,color[i].tolist(),-1)

        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)

        traffic_optical_flow.write(cv2.add(frame, mask))
        
    else:
        break

traffic_optical_flow.release()

In [14]:
!ffmpeg -hide_banner -loglevel error -i "./processed/movie_optical.avi" -y "./processed/movie_optical.mp4"

'ffmpeg' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
Video.from_file('./processed/movie_optical.mp4')

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\r\x1dmoov\x00\x00\x00lmvhd\x00\x00…